# Bit Manipulation in Julia

In this notebook, we will explore different methods for representing a `visited` array and performing operations on them in Julia. We will use the `Vector{Int}`, and `BitMasking` types for our `visited` array.

We use a vector of `UInt32`. Each UInt32 can represent 32 bits, hence 32 nodes. When there are more than 32 nodes, we need more than one `UInt32` numbers.

In [1]:
const BitMasking = Vector{UInt32} # alias for bit masking

Vector{UInt32} (alias for Array{UInt32, 1})

## Basic Bit Manipulations

In [2]:
n_nodes = 25
v = UInt32(0) # Since n_nodes < 32, we need only one `UInt32` to represent the bit mask

# We visit the node number 1
node = 1
bitmask = UInt32(1) << (node - 1) # left shift: 1 << (node - 1) is equivalent to 2^(node - 1) 

println("v           = ", bitstring(v))
println("bitmask     = ", bitstring(bitmask))
v = v | bitmask # bitwise OR
println("v | bitmask = ", bitstring(v))


v           = 00000000000000000000000000000000
bitmask     = 00000000000000000000000000000001
v | bitmask = 00000000000000000000000000000001


In [8]:
# We visit the node number 5
node = 5
bitmask = UInt32(1) << (node - 1) # left shift: 1 << (node - 1) is equivalent to 2^(node - 1) 

println("v           = ", bitstring(v))
println("bitmask     = ", bitstring(bitmask))
v = v | bitmask # bitwise OR
println("v | bitmask = ", bitstring(v))

v           = 00000000000000000000000000010001
bitmask     = 00000000000000000000000000010000
v | bitmask = 00000000000000000000000000010001


2

In [5]:
# Check if the node number 1 is visited
node = 1
bitmask = UInt32(1) << (node - 1) 

println("v           = ", bitstring(v))
println("bitmask     = ", bitstring(bitmask))
println("v & bitmask = ", bitstring(v & bitmask)) # bitwise AND

isvisited1 = (v & bitmask) != 0


v           = 00000000000000000000000000010001
bitmask     = 00000000000000000000000000000001
v & bitmask = 00000000000000000000000000000001


true

In [6]:
# Check if the node number 10 is visited
node = 10
bitmask = UInt32(1) << (node - 1) 

println("v           = ", bitstring(v))
println("bitmask     = ", bitstring(bitmask))
println("v & bitmask = ", bitstring(v & bitmask)) # bitwise AND

isvisited1 = (v & bitmask) != 0


v           = 00000000000000000000000000010001
bitmask     = 00000000000000000000001000000000
v & bitmask = 00000000000000000000000000000000


false

## `isvisited` function

The `isvisited` function checks if a node has been visited. It takes a `visited` array and a `node` as input and returns `true` if the node has been visited and `false` otherwise.

Here's how it works for different types of `visited` arrays:


In [7]:
function isvisited(visited::BitMasking, node::Int)
    index = (node - 1) ÷ 32 + 1
    bit = (node - 1) % 32 + 1
    bitmask = UInt32(1) << (bit - 1)
    return (visited[index] & bitmask) != 0
end

function isvisited(visited::Vector{Int}, node::Int)
    return visited[node] != 0
end


isvisited (generic function with 2 methods)

## `markvisited!` function

The `markvisited!` function marks a node as visited. It takes a `visited` array and a `node` as input and sets the corresponding element in the `visited` array to `true` or `1`.

Here's how it works for different types of `visited` arrays:


In [8]:
function markvisited!(visited::BitMasking, node::Int)
    index = (node - 1) ÷ 32 + 1
    bit = (node - 1) % 32 + 1
    visited[index] |= (UInt32(1) << (bit - 1))
end

function markvisited!(visited::Vector{Int}, node::Int)
    visited[node] = 1
end

markvisited! (generic function with 2 methods)

## `isdominating` function

The `isdominating` function checks the dominance rule. It returns false if we cannot conclude that `v1` dominates `v2`, i.e. there exists an `i` such that `v1[i] > v2[i]`. It returns true if `v1` dominates `v2`, i.e., `v1[i] <= v2[i]` for all `i`.

In [9]:
# Return false if there exists an i such that v1[i] > v2[i]
# Return true if v1 dominates v2, i.e., v1[i] <= v2[i] for all i
function isdominating(v1::BitMasking, v2::BitMasking)
    for mask_idx in eachindex(v1)
        if (v1[mask_idx] & v2[mask_idx]) != v1[mask_idx]
            return false
        end
    end
    return true
end

isdominating(v1::Vector{Int}, v2::Vector{Int}) = !any(v1 .> v2)


isdominating (generic function with 2 methods)

To see how `isdominating()` function works for `BitMasking`, see the example below.

In [10]:
v1 = UInt32(13) # visited nodes: 1, 3, 4
v2 = UInt32(15) # visited nodes: 1, 2, 3, 4
println("v1      = ", bitstring(v1))
println("v2      = ", bitstring(v2))
println("v1 & v2 = ", bitstring(v1 & v2)) #bitwise AND, representing the nodes visited by both v1 and v2

if v1 & v2 == v1
    println("v1 is a subset of v2, i.e. v1[i] <= v2[i] ∀ i")
else 
    println("v1 is not a subset of v2, i.e. v1[i] > v2[i] for some i")
end

v1      = 00000000000000000000000000001101
v2      = 00000000000000000000000000001111
v1 & v2 = 00000000000000000000000000001101
v1 is a subset of v2, i.e. v1[i] <= v2[i] ∀ i


## Tests

In [18]:
using BenchmarkTools

In [19]:
n_nodes = 500

idx = rand(1:n_nodes)

visited_bitmask = zeros(UInt32, (n_nodes + 31) ÷ 32) # Number of integers needed for the bitmask
visited_intvector = zeros(Int, n_nodes)

println("---------------------------------------------------------------")
println("Comparing the initialization operation")
print("BitMsking:"); @btime visited_bitmask = zeros(UInt32, ($n_nodes + 31) ÷ 32) # Number of integers needed for the bitmask
print("IntVector:"); @btime visited_intvector = zeros(Int, $n_nodes)

println("---------------------------------------------------------------")
println("Comparing the update operation: markvisited!(visited, i)")
print("BitMsking:"); @btime markvisited!($visited_bitmask, $idx)
print("IntVector:"); @btime markvisited!($visited_intvector, $idx)

println("---------------------------------------------------------------")
println("Comparing the check-if-visited operation: isvisited(visited, i)")
print("BitMsking:"); @btime isvisited($visited_bitmask, $idx)
print("IntVector:"); @btime isvisited($visited_intvector, $idx)    
println("---------------------------------------------------------------")

---------------------------------------------------------------
Comparing the initialization operation
BitMsking:  17.618 ns (1 allocation: 128 bytes)
IntVector:  194.400 ns (1 allocation: 4.06 KiB)
---------------------------------------------------------------
Comparing the update operation: markvisited!(visited, i)
BitMsking:  3.155 ns (0 allocations: 0 bytes)
IntVector:  2.113 ns (0 allocations: 0 bytes)
---------------------------------------------------------------
Comparing the check-if-visited operation: isvisited(visited, i)
BitMsking:  2.524 ns (0 allocations: 0 bytes)
IntVector:  1.853 ns (0 allocations: 0 bytes)
---------------------------------------------------------------


`BitMasking` is the slowest for `markvisited!()` and `isvisited()`.

Let's see how `isdominating()` does.

In [20]:
# Prepare for comparison

# Bitmasking
num_ints = (n_nodes + 31) ÷ 32
visited1_bitmasking = zeros(UInt32, num_ints)
visited2_bitmasking = zeros(UInt32, num_ints)
visited3_bitmasking = zeros(UInt32, num_ints)

# Intvector
visited1_intvector = zeros(Int, n_nodes)
visited2_intvector = zeros(Int, n_nodes)
visited3_intvector = zeros(Int, n_nodes)

# Mark some nodes as visited 
set_nodes = unique(rand(1:n_nodes, round(Int, n_nodes / 2)))
for node in set_nodes
    markvisited!(visited1_bitmasking, node)
    markvisited!(visited2_bitmasking, node)

    markvisited!(visited1_intvector, node)
    markvisited!(visited2_intvector, node)
end

# Mark some nodes as visited only in the second set
# so that the first set is a subset of the second set
for i in 1:n_nodes/10
    node = rand(1:n_nodes)
    markvisited!(visited2_bitmasking, node)
    markvisited!(visited2_intvector, node)
end
# Mark some other nodes as visited only in the third set 
# so that the third set is not a subset of the second set
for i in 1:n_nodes/10
    node = rand(1:n_nodes)
    markvisited!(visited3_bitmasking, node)
    markvisited!(visited3_intvector, node)
end




In [21]:
println("[[ When visited1 is a subset of visited2 ]]")
println("Comparing the dominance check operation")
print("BitMsking :"); @btime isdominating($visited1_bitmasking, $visited2_bitmasking);
print("IntVector :"); @btime isdominating($visited1_intvector, $visited2_intvector);

[[ When visited1 is a subset of visited2 ]]
Comparing the dominance check operation
BitMsking :  8.635 ns (0 allocations: 0 bytes)
IntVector :  313.877 ns (3 allocations: 4.34 KiB)


In [22]:
println("[[ When visited1 is not a subset of visited2 ]]")
print("BitMsking :"); @btime isdominating($visited3_bitmasking, $visited2_bitmasking);
print("IntVector :"); @btime isdominating($visited3_intvector, $visited2_intvector);

[[ When visited1 is not a subset of visited2 ]]
BitMsking :  2.695 ns (0 allocations: 0 bytes)
IntVector :  359.210 ns (3 allocations: 4.34 KiB)
